In [ ]:
include("src//graph.jl")
include("src//tokens.jl")
include("src//search.jl")

using ..HllGraph
using ..HllSets
using ..Tokens
using ..Search

using Redis
using EasyConfig
using UUIDs
using Random

conn = Redis.RedisConnection()

In [ ]:
# Initialize test HllSets
hll1 = HllSets.HllSet{10}()
hll2 = HllSets.HllSet{10}()
hll3 = HllSets.HllSet{10}()
hll4 = HllSets.HllSet{10}()
hll5 = HllSets.HllSet{10}()

# Generate datasets from random strings
s1 = Set(randstring(7) for _ in 1:10)
s2 = Set(randstring(7) for _ in 1:15)
s3 = Set(randstring(7) for _ in 1:100)
s4 = Set(randstring(7) for _ in 1:20)
s5 = Set(randstring(7) for _ in 1:130)

# Add datasets to HllSets
HllSets.add!(hll1, s1)
HllSets.add!(hll2, s2)
HllSets.add!(hll3, s3)
HllSets.add!(hll4, s4)
HllSets.add!(hll5, s5)

print(hll1)

In [ ]:
# Pr9nt cardinality of datasets and HllSets side by side
print(length(s1), " : ", HllSets.count(hll1), "\n")
print(length(s2), " : ", HllSets.count(hll2), "\n")
print(length(s3), " : ", HllSets.count(hll3), "\n")
print(length(s4), " : ", HllSets.count(hll4), "\n")
print(length(s5), " : ", HllSets.count(hll5), "\n\n")

# union
print(length(s1 ∪ s2 ∪ s3 ∪ s4 ∪ s5), " : ", HllSets.count(hll1 ∪ hll2 ∪ hll3 ∪ hll4 ∪ hll5), "\n")

# intersection
print(length(s1 ∩ s2 ∩ s3 ∩ s4 ∩ s5), " : ", HllSets.count(hll1 ∩ hll2 ∩ hll3 ∩ hll4 ∩ hll5), "\n")

In [ ]:
node1 = HllGraph.Node(HllSets.id(hll1), ["node1"], HllSets.dump(hll1))
node2 = HllGraph.Node(HllSets.id(hll2), ["node2"], HllSets.dump(hll2))
node3 = HllGraph.Node(HllSets.id(hll3), ["node3"], HllSets.dump(hll3))
node4 = HllGraph.Node(HllSets.id(hll4), ["node4"], HllSets.dump(hll4))
node5 = HllGraph.Node(HllSets.id(hll5), ["node5"], HllSets.dump(hll5))

# println(node1.searchable)

graph = HllGraph.Graph([node1, node2, node3, node4, node5], [])

In [ ]:
using JSON3

HllGraph.set_node(conn, node1, "b")
HllGraph.set_node(conn, node2, "b")
HllGraph.set_node(conn, node3, "b")
HllGraph.set_node(conn, node4, "b")
HllGraph.set_node(conn, node5, "b")
# println(JSON3.read(s1, Set))
Tokens.set_tokens(conn, s1, node1.sha1, "b")
Tokens.set_tokens(conn, s2, node2.sha1, "b")
Tokens.set_tokens(conn, s3, node3.sha1, "b")
Tokens.set_tokens(conn, s4, node4.sha1, "b")
Tokens.set_tokens(conn, s5, node5.sha1, "b")

In [ ]:
# BLOB = bin2hex(Search.vec_query_tokens("7cc4390f6b8ac3988b658793e4fe5871bfa20564", 
#         "98bc939b49f3b0cf21bceb09463bb8927a265ca4"))

BLOB = (Search.vec_query_tokens("f6a088fc217f866b0e5e46b9375afe202dfe6b62", 
        "4e73a00392d396fb1d0fa64f3a58ba01925931ad"))

# println(BLOB)

# query = "(*)=>[KNN 100 @searchable \$query_vector]"

# # println(length(BLOB))
# vect = ["FT.SEARCH", "tokens","$query", "PARAMS", "2", "query_vector", "$BLOB", 
#         "SORTBY", "__searchable_score", "ASC", "RETURN", "2", "id", "token", "DIALECT", "2"]

# # println(vect)

# knn = "(*)=>[KNN 100 @searchable" * raw"$query_vector" * "]"

# q_vector = ["FT.SEARCH", "tokens", "$knn", 
#                 "PARAMS", "2", "query_vector", "$BLOB", "SORTBY", "__searchable_score", "ASC", "RETURN", "2", "id", "token", "DIALECT", "2"]

# println(q_vector)

FT_SEARCH = """
FT.SEARCH tokens "(*)=>[KNN 100 @searchable \$query_vector]" PARAMS 2 "query_vector" "$" SORTBY "__searchable_score" ASC RETURN 2 "id" "token" DIALECT 2
"""

search_result = Redis.execute_command(conn, q_vector)    
    # "PARAMS", "2", "BLOB", "\x12\xa9\xf5\x6c", 
    # )

B
# println(search_result)

In [ ]:
FT_SEARCH_TEMPLATE = """
FT.SEARCH tokens "(*)=>[KNN \$knn_count @searchable \$query_vector]" PARAMS \$param_count "query_vector" "\$_query_vector" SORTBY "__searchable_score" ASC RETURN 2 "id" "token" DIALECT 2
"""

# Function to generate the FT.SEARCH command
function generate_ft_search(knn_count, param_count, query_vector)
    return replace(FT_SEARCH_TEMPLATE, 
                   "\$knn_count" => string(knn_count), 
                   "\$param_count" => string(param_count), 
                   "\$_query_vector" => query_vector)
end

# Example usage
knn_count = 100
param_count = 2
query_vector = "00000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000100000000000000000000"

FT_SEARCH = generate_ft_search(knn_count, param_count, query_vector)

println(FT_SEARCH) 

In [ ]:
using Strs
using ASCIIStr

# Example usage
n_n = 100
param_count = 2
query_vector = "00000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000100000000000000000000"

search_query = f"\"(*)=>[KNN \(n_n) @searchable $query_vector]\""
params_1 = f"\"query_vector\""
params_2 = f"\"\(query_vector)\""
_sortby = ASCIStr("""SORTBY __searchable_score ASC""")
#  ASC RETURN 2 \"id\" \"token\" DIALECT 2"

println(["FT.SEARCH", "tokens", search_query, "PARAMS", "2", 
                params_1, params_2, 
                #_sortby, #"ASC", 
                "RETURN", "2", f"\"id\"", f"\"token\"", "DIALECT", "2"])

search_result = Redis.execute_command(conn, ["FT.SEARCH", "tokens", search_query, "PARAMS", "2", 
                params_1, params_2, 
                #_sortby, #"ASC", 
                "RETURN", "2", f"\"id\"", f"\"token\"", "DIALECT", "2"])    
    # "PARAMS", "2", "BLOB", "\x12\xa9\xf5\x6c", 
    # ) 

In [ ]:
using Strs

# Example usage
n_n = 100
param_count = 2
query_vector = "00000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000100000000000000000000"


FT_SEARCH = "FT.SEARCH"
idx_name = "tokens"
_params = "PARAMS"
param_num = "2"
search_query = f""" "(*)=>[KNN \(n_n) @searchable $query_vector]" """
params_1 = raw""" "query_vector" """
params_2 = f""" "\(query_vector)" """
_sortby = ASCIIStr("SORTBY \"__searchable_score\" ASC")
_return = "RETURN"
_ret_num = "2"
_ret_1 = f"\"id\""
_ret_2 = f"\"token\""
_dialect = "DIALECT"
_dialect_num = "2"

println([FT_SEARCH, idx_name, search_query, _params, param_num, 
                params_1, params_2, 
                _sortby, #"ASC", 
                _return, _ret_num, _ret_1, _ret_2, _dialect, _dialect_num])

search_result = Redis.execute_command(conn, [FT_SEARCH, idx_name, search_query, _params, param_num, 
                params_1, params_2, 
                _sortby, #"ASC", 
                _return, _ret_num, _ret_1, _ret_2, _dialect, _dialect_num])    
    


In [ ]:
using PyCall

# Import a Python module
math = pyimport("math")

# Call a Python function
result = math.sqrt(16)
println(result)  # Output: 4.0

# Execute arbitrary Python code
py"""
import numpy as np
a = np.array([1, 2, 3])
b = np.array([4, 5, 6])
c = a + b
"""

# Retrieve the result from Python
c = py"c"
println(c)  # Output: [5 7 9]

In [2]:
using PyCall


query_vector = "00000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000100000000000000000000"

redis = pyimport("redis")
query = pyimport("redis.commands.search.query")

r = redis.Redis(host="localhost", port=6379, db=0)
r.set("foo", "bar")

_query = query.Query(raw"(*)=>[KNN 3 @searchable $vec_param]").sort_by("__searchable_score").dialect(2)


println(_query.query_string)

ft = r.ft("tokens")
res = ft.search(_query, query_params = ("vec_param", query_vector))

print_results(res)

# # Import the Python script as a module
# redis_script = pyimport("redis_script")

# # Define the Redis command
# command = ["FT.SEARCH", "tokens", "(*)=>[KNN 100 @searchable \$query_vector]", "PARAMS", "2", 
#            "query_vector", "00000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000100000000000000000000", 
#            "RETURN", "2", "id", "token", "DIALECT", "2"]

# # Execute the Redis command using the Python script
# search_result = redis_script.execute_redis_command(command)

# println(search_result)

PyObject <bound method Query.query_string of <redis.commands.search.query.Query object at 0x7f7105db14f0>>


LoadError: PyError ($(Expr(:escape, :(ccall(#= /home/alexmy/.julia/packages/PyCall/1gn3u/src/pyfncall.jl:43 =# @pysym(:PyObject_Call), PyPtr, (PyPtr, PyPtr, PyPtr), o, pyargsptr, kw))))) <class 'AttributeError'>
AttributeError("'tuple' object has no attribute 'items'")
  File "/home/alexmy/.local/lib/python3.12/site-packages/redis/commands/search/commands.py", line 499, in search
    args, query = self._mk_query_args(query, query_params=query_params)
                  ^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^
  File "/home/alexmy/.local/lib/python3.12/site-packages/redis/commands/search/commands.py", line 479, in _mk_query_args
    args += self.get_params_args(query_params)
            ^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^
  File "/home/alexmy/.local/lib/python3.12/site-packages/redis/commands/search/commands.py", line 462, in get_params_args
    for key, value in query_params.items():
                      ^^^^^^^^^^^^^^^^^^


In [ ]:
println(HllGraph.get_node(conn, node1.sha1, "b"))
println(node1.sha1)
println(node1.labels)

nodes = HllGraph.get_nodes(conn, "b")
edges = HllGraph.get_edges(conn, "b")
println(nodes)

println(edges)

HllGraph.Graph(nodes, edges)

In [ ]:
union_node = HllGraph.union_nodes(conn, node1, node2, ["union"], "b")

In [ ]:
println(HllGraph.get_edges(conn, "b"))

In [ ]:
value ="0000000000010000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000"

println(length(value))